In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 102.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.0 MB/s eta 0:00:00


In [2]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.0 MB/s eta 0:00:00


In [3]:
pip install bertviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.0 MB/s eta 0:00:00


In [4]:
from transformers import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)
text = "time flies like an arrow"
show(model, 'bert', tokenizer, text, display_mode="light", layer=0, head=8)

Output hidden; open in https://colab.research.google.com to view.

In [5]:
inputs = tokenizer(text, return_tensors='pt', add_special_tokens=False)
inputs.input_ids

tensor([[ 2051, 10029,  2066,  2019,  8612]])

In [6]:
from torch import nn
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

Embedding(30522, 768)

In [7]:
config.vocab_size

30522

In [8]:
inputs_embs = token_emb(inputs.input_ids)
inputs_embs.size()

torch.Size([1, 5, 768])

In [9]:
import torch
from math import sqrt

query = key = value = inputs_embs
dim_k = key.size(-1)
scores = torch.bmm(query, key.transpose(1,2))/sqrt(dim_k)
scores.size()

torch.Size([1, 5, 5])

In [10]:
key.transpose(1,2).size()

torch.Size([1, 768, 5])

In [11]:
import torch.nn.functional as F

weights = F.softmax(scores, dim=-1)
weights.sum(dim=-1)

tensor([[1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)

In [12]:
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

torch.Size([1, 5, 768])

In [13]:
def scaled_dot_product_attention(query, key, value):
  dim_k = query.size(-1)
  scores = torch.bmm(query, key.transpose(1,2))/sqrt(dim_k)
  weights = F.softmax(scores, dim=-1)
  return torch.bmm(weights, value)

In [14]:
word_idx = {w:i for i, w in enumerate(text.split(" "))}
for w1 in word_idx.keys():
  for w2 in word_idx.keys():
    simil = torch.dot(attn_outputs[0][word_idx[w1]], attn_outputs[0][word_idx[w2]])
    print(f"{w1}, {w2}의 유사도:", simil)

time, time의 유사도: tensor(688.1329, grad_fn=<DotBackward0>)
time, flies의 유사도: tensor(52.2416, grad_fn=<DotBackward0>)
time, like의 유사도: tensor(14.0394, grad_fn=<DotBackward0>)
time, an의 유사도: tensor(-3.1775, grad_fn=<DotBackward0>)
time, arrow의 유사도: tensor(-13.1436, grad_fn=<DotBackward0>)
flies, time의 유사도: tensor(52.2416, grad_fn=<DotBackward0>)
flies, flies의 유사도: tensor(746.8833, grad_fn=<DotBackward0>)
flies, like의 유사도: tensor(3.4443, grad_fn=<DotBackward0>)
flies, an의 유사도: tensor(-9.9430, grad_fn=<DotBackward0>)
flies, arrow의 유사도: tensor(-6.9201, grad_fn=<DotBackward0>)
like, time의 유사도: tensor(14.0394, grad_fn=<DotBackward0>)
like, flies의 유사도: tensor(3.4443, grad_fn=<DotBackward0>)
like, like의 유사도: tensor(719.2166, grad_fn=<DotBackward0>)
like, an의 유사도: tensor(-30.5133, grad_fn=<DotBackward0>)
like, arrow의 유사도: tensor(-2.7751, grad_fn=<DotBackward0>)
an, time의 유사도: tensor(-3.1775, grad_fn=<DotBackward0>)
an, flies의 유사도: tensor(-9.9430, grad_fn=<DotBackward0>)
an, like의 유사도: tensor(-30.

In [15]:
class AttentionHead(nn.Module):
  def __init__(self, embed_dim, head_dim):
    super().__init__()
    self.q = nn.Linear(embed_dim, head_dim)
    self.k = nn.Linear(embed_dim, head_dim)
    self.v = nn.Linear(embed_dim, head_dim)

  def forward(self, hidden_state):
    attn_outputs = scaled_dot_product_attention(
        self.q(hidden_state), self.k(hidden_state), self.v(hidden_state)
    )
    return attn_outputs

In [16]:
class MultiHeadAttention(nn.Module):
  def __init__(self, config):
    super().__init__()
    embed_dim = config.hidden_size
    num_heads = config.num_attention_heads
    head_dim = embed_dim // num_heads
    self.heads = nn.ModuleList(
        [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)]
    )
    self.output_linear = nn.Linear(embed_dim, embed_dim)

  def forward(self, hidden_state):
    x = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
    x = self.output_linear(x)
    return x

In [17]:
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(inputs_embs)
attn_output.size()

torch.Size([1, 5, 768])

In [18]:
word_idx = {w:i for i, w in enumerate(text.split(" "))}
for w1 in word_idx.keys():
  for w2 in word_idx.keys():
    simil = torch.dot(attn_output[0][word_idx[w1]], attn_output[0][word_idx[w2]])
    print(f"{w1}, {w2}의 유사도:", simil)

time, time의 유사도: tensor(18.3117, grad_fn=<DotBackward0>)
time, flies의 유사도: tensor(17.9118, grad_fn=<DotBackward0>)
time, like의 유사도: tensor(17.7289, grad_fn=<DotBackward0>)
time, an의 유사도: tensor(17.0865, grad_fn=<DotBackward0>)
time, arrow의 유사도: tensor(17.2508, grad_fn=<DotBackward0>)
flies, time의 유사도: tensor(17.9118, grad_fn=<DotBackward0>)
flies, flies의 유사도: tensor(19.4932, grad_fn=<DotBackward0>)
flies, like의 유사도: tensor(18.0130, grad_fn=<DotBackward0>)
flies, an의 유사도: tensor(18.0924, grad_fn=<DotBackward0>)
flies, arrow의 유사도: tensor(18.0420, grad_fn=<DotBackward0>)
like, time의 유사도: tensor(17.7289, grad_fn=<DotBackward0>)
like, flies의 유사도: tensor(18.0130, grad_fn=<DotBackward0>)
like, like의 유사도: tensor(19.1973, grad_fn=<DotBackward0>)
like, an의 유사도: tensor(17.8421, grad_fn=<DotBackward0>)
like, arrow의 유사도: tensor(17.8209, grad_fn=<DotBackward0>)
an, time의 유사도: tensor(17.0865, grad_fn=<DotBackward0>)
an, flies의 유사도: tensor(18.0924, grad_fn=<DotBackward0>)
an, like의 유사도: tensor(17.8421

In [19]:
from bertviz import head_view
from transformers import AutoModel

model = AutoModel.from_pretrained(model_ckpt, output_attentions=True)

sentence_a = "time flies like an arrow"
sentence_b = "fruit flies like a banana"

viz_inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt')
attention = model(**viz_inputs).attentions
sentence_b_start = (viz_inputs.token_type_ids == 0).sum(dim=1)
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])

head_view(attention, tokens, sentence_b_start, heads=[8])

Output hidden; open in https://colab.research.google.com to view.

In [20]:
def make_attention_output(q,k,v):
  dim_k = k.size(-1)
  score = torch.bmm(q,k.transepose(1,2))/sqrt(dim_k)
  score = F.softmax(score, dim=-1) #열,행에서의 적용 여부 판단
  return torch.bmm(score,v)

In [21]:
viz_inputs

{'input_ids': tensor([[  101,  2051, 10029,  2066,  2019,  8612,   102,  5909, 10029,  2066,
          1037, 15212,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [22]:
#mulit-head attention 연습
class one_head(nn.Module):
  def __init__(self, in_d, out_d):
    super().__init__()
    self.q = nn.Linear(in_d, out_d)
    self.k = nn.Linear(in_d, out_d)
    self.v = nn.Linear(in_d, out_d)

  def forward(self, hidden_state):
    atten_out = make_attention_output(self.q(hidden_state), self.k(hidden_state)\
                          ,self.v(hidden_state))

    return atten_out

class multi_head(nn.Module):
  def __init__(self,config):
    super().__init__()
    in_d = config.hidden_size
    num_head = config.num_attention_heads
    out_d = in_d//num_head
    self.head = nn.ModuleList(
        [one_head(in_d, out_d) for _ in range(num_head)]
    )
    self.lastlayer = nn.Linear(in_d, in_d)

  def forward(self, hidden_state):
    all_head = torch.cat([h(hidden_state) for h in self.head], axis=-1)
    result = self.lastlayer(all_head)
    return result

In [23]:
class FeedForward(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.linear_1 = nn.Linear(config.hidden_size, config.intermediate_size)
    self.linear_2 = nn.Linear(config.intermediate_size, config.hidden_size)
    self.gelu = nn.GELU()
    self.dropout = nn.Dropout(config.hidden_dropout_prob)

  def forward(self, x):
    x = self.linear_1(x)
    x = self.gelu(x)
    x = self.linear_2(x)
    x = self.dropout(x)
    return x

In [24]:
feed_forward = FeedForward(config)
ff_outputs = feed_forward(attn_outputs) #알아서 768개가 들어가는건가...
ff_outputs.size()

torch.Size([1, 5, 768])

In [25]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.layer_norm_1 = nn.LayerNorm(config.hidden_size)
    self.layer_norm_2 = nn.LayerNorm(config.hidden_size)
    self.attention = MultiHeadAttention(config)
    self.feed_forward = FeedForward(config)

  def forward(self, x):
    hidden_state = self.layer_norm_1(x)
    x = x + self.attention(hidden_state)
    x = x + self.feed_forward(self.layer_norm_2(x))
    return x

In [26]:
encoder_layer = TransformerEncoderLayer(config)
encoder_output = encoder_layer(inputs_embs)
encoder_output.size()

torch.Size([1, 5, 768])

In [27]:
inputs_embs.size()

torch.Size([1, 5, 768])

In [28]:
class Embeddings(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.token_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
    self.position_embeddings = nn.Embedding(config.max_position_embeddings,\
                                            config.hidden_size)
    self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
    self.dropout = nn.Dropout()

  def forward(self, input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long).unsqueeze(0)
    token_embeddings = self.token_embeddings(input_ids)
    position_embeddings = self.position_embeddings(position_ids)
    embeddings = token_embeddings + position_embeddings
    embeddings = self.layer_norm(embeddings)
    embeddings = self.dropout(embeddings)
    return embeddings

embedding_layer = Embeddings(config)
embedding_layer(inputs.input_ids).size()

torch.Size([1, 5, 768])

In [29]:
class TransformerEncoder(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.embeddings = Embeddings(config)
    self.layers = nn.ModuleList([TransformerEncoderLayer(config)\
                                 for _ in range(config.num_hidden_layers)])
  def forward(self, x):
    x = self.embeddings(x)
    for layer in self.layers:
      x = layer(x)
    return x

In [30]:
encoder = TransformerEncoder(config)
encoder(inputs.input_ids).size()

torch.Size([1, 5, 768])

In [31]:
#분류 헤드 추가
class TransformerForSequenceClassification(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.encoder = TransformerEncoder(config)
    self.dropout = nn.Dropout(config.hidden_dropout_prob)
    self.classifier = nn.Linear(config.hidden_size, config.num_labels)
  def forward(self, x):
    x = self.encoder(x)[:,0,:]
    x = self.dropout(x)
    x = self.classifier(x)
    return x

In [32]:
config.num_labels = 3
encoder_classifier = TransformerForSequenceClassification(config)
encoder_classifier(inputs.input_ids).size()

torch.Size([1, 3])

In [37]:
encoder_classifier(inputs.input_ids)

tensor([[-0.4486,  0.7169,  1.4553]], grad_fn=<AddmmBackward0>)

In [49]:
seq_len = inputs.input_ids.size(-1)
mask = torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0)
mask[0]

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [54]:
scores.masked_fill(mask == 0, float("-inf"))

tensor([[[24.8309,    -inf,    -inf,    -inf,    -inf],
         [ 1.8851, 26.9508,    -inf,    -inf,    -inf],
         [ 0.5066,  0.1243, 25.9525,    -inf,    -inf],
         [-0.1147, -0.3588, -1.1011, 27.4130,    -inf],
         [-0.4743, -0.2497, -0.1001,  0.8766, 28.3419]]],
       grad_fn=<MaskedFillBackward0>)

In [46]:
def scaled_dot_product_attention(query, key, value, mask=None):
  dim_k = query.size(-1)
  scores = torch.bmm(query, key.transpose(1,2))/sqrt(dim_k)
  if mask is not None:
    scores = scores.masked_fill(mask == mask, float('-inf'))
  weights = F.softmax(scores, dim=-1)
  return weights.bmm(value)